In [6]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import nyu_dataset
import pix2pixHD_model
from torch.utils.data import DataLoader 
from sklearn.cluster import KMeans

In [7]:
_cuda = 2

In [8]:
encoder = pix2pixHD_model.FeatureEncoder(3, 3, instance_norm=True).cuda(_cuda)

In [9]:
encoder.load_state_dict(torch.load('./pix2pixHD/models/encoder_100_0'))

In [10]:
segmentation_dataset_validation = nyu_dataset.SegmentationDataset(path_to_datafolder='./datasets/nyu/val/', 
    transforms=nyu_dataset.SegmentationTransform(train_trainsforms=False, resize=False), 
    use_instance_segmentation=True)
data_loader_val = DataLoader(segmentation_dataset_validation, batch_size=1, shuffle=False, num_workers=1)

In [20]:
dictionary = {}

count = 0
for image, segm, inst, bord in data_loader_val:
    image = Variable(image.cuda(_cuda), requires_grad=False, volatile=True)
    segm = Variable(segm.cuda(_cuda), requires_grad=False, volatile=True)
    inst = inst.cuda(_cuda)
    count += 1
    print(count)
    if count > 100:
        break 
    
    
    encoded = encoder.forward(image, inst)

    array_segm = segm.data.cpu().numpy()
    array_encoded = encoded.data.cpu().numpy()
    classes = range(len(array_segm[0]))

    for k in classes:
                    
        idx = np.argwhere(array_segm[0][k]==1)

        for item in idx:
            x = item[0].tolist()
            y = item[1].tolist()
            
            feature_1 = array_encoded[0][0][x,y]
            feature_2 = array_encoded[0][1][x,y]
            feature_3 = array_encoded[0][2][x,y]
            
            feature_array = [feature_1,feature_2,feature_3]

            if k in dictionary:
                if feature_array == any (dictionary[k]):
                     pass
                    
                else:
                    dictionary[k].append(feature_array)

            else:   
                 dictionary[k] = (feature_array)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [21]:
dictionary

{0: [-0.5309301,
  0.99909556,
  0.99951905,
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.99951905],
  [-0.5309301, 0.99909556, 0.999519

In [25]:
cluster_centres = dict.fromkeys (classes, [])
for i in dictionary:
    kmeans = KMeans(n_clusters = 10).fit( )
    k_centers = kmeans.cluster_centers_
    cluster_centres[i].append(k_centers)

TypeError: fit() takes from 2 to 3 positional arguments but 4 were given

## Number of instance and number of classes

In [ ]:
i =0 
for image, segm, inst, bord in data_loader_val:
    image = Variable(image.cuda(_cuda), requires_grad=False, volatile=True)
    segm = Variable(segm.cuda(_cuda), requires_grad=False, volatile=True)
    inst = inst.cuda(_cuda)
    i += 1
    if i == 25:
        break

In [ ]:
classes_map = np.argmax(segm[0].data.cpu().numpy(), axis=0)

instance = inst.cpu().numpy()

print(len(np.unique(classes_map)))
print(len(np.unique(instance)))

## Right loop

In [ ]:
encoded = encoder.forward(image, inst)

array_segm = segm.data.cpu().numpy()
array_encoded = encoded.data.cpu().numpy()
classes = range(len(array_segm[0]))
dictionary = {}
for k in classes:

    idx = np.argwhere(array_segm[0][k]==1)

    for item in idx:
        x = item[0].tolist()
        y = item[1].tolist()

        feature_1 = array_encoded[0][0][x,y]
        feature_2 = array_encoded[0][1][x,y]
        feature_3 = array_encoded[0][2][x,y]
        feature_array = np.asarray([feature_1,feature_2,feature_3])

        if k in dictionary:
            if feature_array.all() in dictionary[k]:
                    pass

            else:
                np.concatenate((dictionary[k], feature_array))

        else:   
                dictionary[k] = (feature_array)


In [ ]:
dictionary

## Garbage

In [ ]:
encoded = encoder.forward(image, inst)

In [ ]:
array_encoded = encoded.data.cpu().numpy()
array_encoded[0][0]

In [ ]:
l = np.array([1, 2, 3])

In [ ]:
dictionary = dict.fromkeys(range(0,3),[1,2,3])
dictionary[4] = [1,2,3,4], [1,2,4]

In [ ]:
if l.all() in dictionary[4]:
    dictionary[5] = 100
else:
    dictionary[5] = -1
    

In [ ]:
dictionary

In [ ]:
np.unique(array_encoded[0][0])

In [ ]:
dictionary[1]

In [ ]:
plt.imshow(np.rollaxis(encoded[0].data.cpu().numpy(), 0,3))

## Old loop

In [ ]:
inst_unique = np.unique(inst)
for i in inst_unique:
    indices = (inst == int(i)).nonzero()
print(inst)

In [ ]:
classes = np.unique(classes_map)

In [ ]:
classes_map = np.argmax(segm[0].data.cpu().numpy(), axis=0)
classes = np.unique(classes_map) 


for idx in range(len(indices[:,0])):
    dictionary = dict.fromkeys (classes, [])
    coord1, coord2 = indices[idx, 2], indices[idx, 3]
    features_vector = encoded[0][:, coord1, coord2]
    one_class = classes_map[coord1, coord2]
    
    
    if one_class in dictionary:
        dictionary[one_class].append(features_vector.data.cpu().numpy().tolist())
    
    else:
        dictionary[one_class] = features_vector



In [ ]:
len(classes_map.flatten())

In [ ]:
print(dictionary)

In [ ]:
classes_map[indices[0, 2], indices[0, 3]]

In [ ]:
if line[0] in years_dict:
        # append the new number to the existing array at this slot
        years_dict[line[0]].append(line[1])
    else:
        # create a new array in this slot
        years_dict[line[0]] = [line[1]]

In [ ]:
# class_names = scipy.io.loadmat('./classMapping40.mat') ['className'][0]

In [ ]:
for j in range(3):
        output_inst = encoded[indices[:, 0], indices[:, 1] + j, indices[:, 2],
                                indices[:, 3]]